In [1]:
import argparse
import random
from gtd.io import save_stdout
from gtd.log import set_log_level
from gtd.utils import Config
from textmorph.edit_model.training_run import EditTrainingRuns

from textmorph.edit_model.editor import EditExample
set_log_level('DEBUG')

from textmorph import data

# create experiment
experiments = EditTrainingRuns(check_commit=False)

In [2]:
# takes in data from original format and break into sequence length 20
def parse(path, c, TYPE, t):
    NOTES_PER_LINE=20
    clean = []
    with open(path) as f:
        for line in f:
            for item in line.lstrip().rsplit():
                clean.append(item)

    with open(key+'_'+TYPE+'_'+t+'_parsed.txt', 'w') as f:
        count = 0
        for item in clean:
            if count < NOTES_PER_LINE-1:
                f.write(item+' ')
                count+=1
            else:
                f.write(item+'\n')
                count = 0
                
    return key+'_'+TYPE+'_'+t+'_parsed.txt'

In [13]:

# TYPE = [mono, poly, poly_dur]
TYPE='mono'
print 'TYPE ', TYPE

# create map of the composer checkpoints
composer={'bach':63, 'beethoven':62, 'handel':64, 'mozart':65, 'brahms':28, 'vivaldi':29, 'schubert':30, 'haydn':31}

for key in composer:
    # set checkpoint
    EDIT_RUN=composer[key]
    
    # create an experiment from saved checkpoint
    exp = experiments.get(EDIT_RUN)
    editor = exp.editor
    print 'Processing ', key, '...'

    # for both test data sets: holdout and split
    test=['holdout', 'split']
    for t in test:
        print '\t'+'Processing ', t
        path = key+'_'+TYPE+'_'+t+'.txt'
        
        # split into seqlen=20
        new_path = parse(path, key, TYPE, t)
        # path of results file
        write_path = key+'_'+TYPE+'_'+t+'_results'+'.txt'
        
        # generate results per line and write to file
        with open(write_path, 'w') as wp:
            with open(new_path) as f:
                count = 0
                for line in f.readlines():
                    
                    # print current line being processed
                    count += 1
                    print '{0}\r'.format(count),
                    
                    # submit example to edit
                    ex = EditExample(line.split(), [], [], [], [], line.split())
                    output_words_batch, edit_traces = editor.edit([ex], beam_size=8)

                    # get ALL candidate results
                    x=edit_traces[0].decoder_trace.candidates

                    # convert candidates to midi-readable format
                    # ie. 'd4 f5 e4 d5 d5 g4 f5 e4 f5 e5 d5 e5 d5 e5 d5 d5 g4 a4 f5 g4 b-5'
                    x=[str(cand)[7:] for cand in x]

                    # choose random candidate of max length
                    max_len=len(max(x, key=len))
                    y=x[random.randint(0, len([i for i in x if len(i)==max_len]))-1]

                    # write to file as a line
                    wp.write(y+'\n')
            wp.close()
            f.close()

TYPE  mono
Reloaded checkpoint #100000



[localhost] local: wc -l /scratch/ly571/neural-editor-data/midi_split/mono/beethoven/train.txt



[localhost] local: wc -l /scratch/ly571/neural-editor-data/midi_split/mono/beethoven/valid.txt



[localhost] local: wc -l /scratch/ly571/neural-editor-data/midi_split/mono/beethoven/test.txt


INFO:root:Reloaded TrainingRun #62


...


[localhost] local: wc -l /scratch/ly571/neural-editor-data/midi_split/mono/mozart/train.txt



[localhost] local: wc -l /scratch/ly571/neural-editor-data/midi_split/mono/mozart/valid.txt



[localhost] local: wc -l /scratch/ly571/neural-editor-data/midi_split/mono/mozart/test.txt


INFO:root:Reloaded TrainingRun #65



Processing  mozart ...
	Processing  holdout
	Processing  split
